# Add Traffic Signals to Network
In this case we already have a traffic signal inventory from the Georgia Department of Transportation, but code for downloading existing traffic signal data from OpenStreetMap (OSM) is also included. This code also retrieves crossings from OSM.

First, we'll find signals in the GDOT data that are not covered in the OSM data. Then we'll take these intersect the them with OSM nodes. Using the reference ID columns on the OSM links, these signals can be added to the links. A turn dataframe is then constructed with all of the links that contain at least one signal.

Once this happens, the link street name will be cross-referenced to validate the match.

- If signals are both road links, check the road name on each to see if it matches the GDOT name
- If non-road link just set it to null for now. Some of these may be crosswalks at the intersection, but they could also be walkways further away from the intersection.

This final result should be QAQC'd.

In [2]:
from pathlib import Path 
import geopandas as gpd
import pandas as pd

from bikewaysim.paths import config
from bikewaysim.network import conflation_tools, modeling_turns

In [ ]:
# use pickled version

In [3]:
links = gpd.read_file(config['network_fp']/'networks.gpkg',layer='osm_links')
nodes = gpd.read_file(config['network_fp']/'networks.gpkg',layer='osm_nodes')
#add attributes back
raw = gpd.read_file(config['osmdwnld_fp']/f"osm.gpkg",layer="raw",ignore_geometry=True)
links = pd.merge(links,raw[['osmid','highway','name']],how='left',on='osmid')
del raw
#create a name col for checking against the GDOT names
links['name0'] = links['name'].apply(lambda x: conflation_tools.contract_suffix(x))

In [18]:
nodes

,osm_N,geometry
0,69123159,POINT (2204210.296 1317129.909)
1,69111202,POINT (2199219.453 1314476.658)
2,69120343,POINT (2199781.589 1315802.120)
3,69097740,POINT (2199758.325 1316067.959)
4,69123846,POINT (2199326.191 1316442.361)
...,...,...
150875,9013734817,POINT (2234182.526 1402113.247)
150876,9013711010,POINT (2234504.087 1402189.711)
150877,7081874786,POINT (2233392.936 1402672.804)
150878,8404063942,POINT (2233691.295 1402702.330)


In [4]:
#buffer function
def buffer_signals(signal_gdf,buffer_ft):
    '''
    Use to create a copy of a gdf and buffer the point geometry
    '''
    signal_gdf = signal_gdf.copy()
    signal_gdf.geometry = signal_gdf.buffer(buffer_ft)
    return signal_gdf

# OSM Signals
No need to conflate, already embedded in the OSM network.

In [5]:
osm_signals = gpd.read_file(config['osmdwnld_fp']/f"osm.gpkg",layer='highway_nodes')
osm_signals = osm_signals[osm_signals['highway']=='traffic_signals']
osm_signals.to_crs(config['projected_crs_epsg'],inplace=True)
osm_signal_ids = set(osm_signals['osmid'].tolist())

In [6]:
signalized_links = links[(links['osm_A'].isin(osm_signal_ids)) | (links['osm_B'].isin(osm_signal_ids))]

## Create turn graph dataframe
_, turns_df = modeling_turns.create_pseudo_dual_graph(signalized_links,'osm_A','osm_B','osm_linkid','oneway')

#add signals ids back in
turns_df['signalized'] = turns_df['source_B'].isin(osm_signal_ids)

#drop unsignalized turn movements
turns_df = turns_df[turns_df['signalized']==True]

#export
turns_df.to_pickle(config['network_fp']/'osm_signals.pkl')

# GDOT Signals

In [7]:
# read gdot signals
keep = ['signalID','mainStreetName','sideStreetName','geometry']
gdot_signals = gpd.read_file(config['gdot_signals_fp'],mask=gpd.read_file(config['studyarea_fp'])).to_crs(links.crs)[keep]
gdot_signals.head()

,signalID,mainStreetName,sideStreetName,geometry
0,1,Luckie St NW,Merritts Ave NW,POINT (2226864.228 1371044.801)
1,343,Airport Loop Rd,Delta Employee Parking,POINT (2218109.221 1318668.699)
2,345,MH Jackson Service Rd (Emergency Signal),Central Cargo Circle,POINT (2222993.259 1324529.642)
3,475,Crosswind Rd,Rental Car Center Pkwy,POINT (2209237.853 1322629.078)
4,486,Rental Car Parkway,West Deck Connector,POINT (2207104.832 1323894.045)


In [8]:
import re
gdot_signals['mainStreetName0'] = gdot_signals['mainStreetName']
gdot_signals['sideStreetName0'] = gdot_signals['sideStreetName'] 
# gdot_signals['mainStreetName0'] = gdot_signals['mainStreetName0'].apply(lambda x: re.sub(r'\(.*?\)', '', x)) # remove anything in parenthesis #TODO instead put / / instead so it can be sperated out
# gdot_signals['sideStreetName0'] = gdot_signals['sideStreetName0'].apply(lambda x: re.sub(r'\(.*?\)', '', x)) # remove anything in parenthesis
gdot_signals['mainStreetName0'] = gdot_signals['mainStreetName0'].apply(lambda x: x.replace('MLK','martin luther king'))#re.sub(r'MLK', 'martin luther king', x)) # change mlk to full version
gdot_signals['sideStreetName0'] = gdot_signals['sideStreetName0'].apply(lambda x: x.replace('MLK','martin luther king'))#re.sub(r'MLK', 'martin luther king', x)) # change mlk to full version
gdot_signals['mainStreetName0'] = gdot_signals['mainStreetName0'].apply(lambda x: x.replace('RDA','ralph david abernathy'))#re.sub(r'RDA', 'ralph david abernathy', x)) # change mlk to full version
gdot_signals['sideStreetName0'] = gdot_signals['sideStreetName0'].apply(lambda x: x.replace('RDA','ralph david abernathy'))# re.sub(r'RDA', 'ralph david abernathy', x)) # change mlk to full version
gdot_signals['mainStreetName0'] = gdot_signals['mainStreetName0'].apply(lambda x: re.sub(r'[()]', '/', x)) # remove anything in parenthesis
gdot_signals['sideStreetName0'] = gdot_signals['sideStreetName0'].apply(lambda x: re.sub(r'[()]', '/', x)) # remove anything in parenthesis
mainStreetName = gdot_signals['mainStreetName0'].apply(lambda x: x.split('/'))
sideStreetName = gdot_signals['sideStreetName0'].apply(lambda x: x.split('/'))
gdot_signals['gdot_names'] = mainStreetName + sideStreetName
gdot_signals['gdot_names'] = gdot_signals['gdot_names'].apply(lambda x: [y for y in x if len(y.replace(' ','')) > 0])
gdot_signals['gdot_names'] = gdot_signals['gdot_names'].apply(lambda list_of_names: [conflation_tools.contract_suffix(name) for name in list_of_names])
gdot_signals.sample(20)

# #some specific regex for the gdot dataset
# import re 

# gdot_signals['mainStreetName0'] = gdot_signals['mainStreetName0'].apply(lambda x: x.lower())
# gdot_signals['mainStreetName0'] = gdot_signals['mainStreetName0'].apply(lambda x: re.sub(r'sr \d+', '', x)) # remove the state routes
# gdot_signals['mainStreetName0'] = gdot_signals['mainStreetName0'].apply(lambda x: re.sub(r'us \d+', '', x)) # remove the us routes
# gdot_signals['mainStreetName0'] = gdot_signals['mainStreetName0'].apply(lambda x: re.sub(r'ga \d+', '', x)) # remove the ga routes
# gdot_signals['mainStreetName0'] = gdot_signals['mainStreetName0'].apply(lambda x: re.sub(r'i-\d+', '', x)) # remove the us routes
# gdot_signals['mainStreetName0'] = gdot_signals['mainStreetName0'].apply(lambda x: re.sub(r'phb \d+', '', x)) # remove the us routes
# gdot_signals['mainStreetName0'] = gdot_signals['mainStreetName0'].apply(lambda x: re.sub(r'\(.*?\)', '', x)) # remove anything in parenthesis
# # gdot_signals['mainStreetName0'] = gdot_signals['mainStreetName0'].apply(lambda x: re.sub('/', ' ', x)) # replace / with space
# gdot_signals['mainStreetName0'] = gdot_signals['mainStreetName0'].apply(lambda x: re.sub('ramp', ' ', x)) # remove ramps from name

# gdot_signals['sideStreetName0'] = gdot_signals['sideStreetName0'].apply(lambda x: x.lower())
# gdot_signals['sideStreetName0'] = gdot_signals['sideStreetName0'].apply(lambda x: re.sub(r'sr \d+', '', x)) # remove the state routes
# gdot_signals['sideStreetName0'] = gdot_signals['sideStreetName0'].apply(lambda x: re.sub(r'us \d+', '', x)) # remove the us routes
# gdot_signals['sideStreetName0'] = gdot_signals['sideStreetName0'].apply(lambda x: re.sub(r'ga \d+', '', x)) # remove the ga routes
# gdot_signals['sideStreetName0'] = gdot_signals['sideStreetName0'].apply(lambda x: re.sub(r'i-\d+', '', x)) # remove the us routes
# gdot_signals['sideStreetName0'] = gdot_signals['sideStreetName0'].apply(lambda x: re.sub(r'phb \d+', '', x)) # remove the us routes
# gdot_signals['sideStreetName0'] = gdot_signals['sideStreetName0'].apply(lambda x: re.sub(r'\(.*?\)', '', x)) # remove anything in parenthesis
# # gdot_signals['sideStreetName0'] = gdot_signals['sideStreetName0'].apply(lambda x: re.sub('/', ' ', x)) # replace / with space
# gdot_signals['sideStreetName0'] = gdot_signals['sideStreetName0'].apply(lambda x: re.sub('ramp', ' ', x)) # remove ramps from name

,signalID,mainStreetName,sideStreetName,geometry,mainStreetName0,sideStreetName0,gdot_names
1221,11308,Lavista Road,Oak Grove Road,POINT (2258174.446 1391265.532),Lavista Road,Oak Grove Road,"[lavista rd, oak grove rd]"
736,7677,SR 154 / Memorial Drive,Boulevard / Carroll Street,POINT (2235351.253 1362956.407),SR 154 / Memorial Drive,Boulevard / Carroll Street,"[sr 154, memorial dr, blvd, carroll st]"
913,8116,Atlanta Student Movement Blvd,James P Brawley Dr SW,POINT (2221772.300 1363946.535),Atlanta Student Movement Blvd,James P Brawley Dr SW,"[atlanta student movement blvd, james p brawle..."
16,23,W Peachtree St,W Peachtree St NW,POINT (2229458.660 1369259.798),W Peachtree St,W Peachtree St NW,"[w peachtree st, w peachtree st nw]"
417,7031,SR 8 / Ponce de Leon Avenue,Barnett Street,POINT (2238226.926 1372815.412),SR 8 / Ponce de Leon Avenue,Barnett Street,"[sr 8, ponce de leon ave, barnett st]"
171,313,Edgewood Ave,Boulevard,POINT (2234131.783 1365727.859),Edgewood Ave,Boulevard,"[edgewood ave, blvd]"
347,48017,Riverdale Rd,Global Gateway Connector,POINT (2204869.181 1322252.176),Riverdale Rd,Global Gateway Connector,"[riverdale rd, global gateway connector]"
1597,452,Lindbergh Way,Lindbergh Drive/MARTA Drwy,POINT (2235237.085 1389977.695),Lindbergh Way,Lindbergh Drive/MARTA Drwy,"[lindbergh wy, lindbergh dr, marta drwy]"
1477,73,Auburn Ave,Fort St,POINT (2232351.598 1366171.746),Auburn Ave,Fort St,"[auburn ave, fort st]"
1395,353,Ralph D Abernathy Blvd,Central St,POINT (2227619.506 1359351.998),Ralph D Abernathy Blvd,Central St,"[ralph d abernathy blvd, central st]"


## Symmetric Difference
Find GDOT signals that are not already covered by OSM. Maybe ask an undergrad to add these into OSM later.

In [9]:
print(gdot_signals.shape[0],'GDOT signals and',osm_signals.shape[0],'OSM signals')
buffer_ft = 100 #selecting 100 ft based on city block sizes and that's about where the number of gdot signals not represented by osm signals drops
buffered_osm_signals = buffer_signals(osm_signals,buffer_ft)
difference = gdot_signals.overlay(buffered_osm_signals,how='difference')
difference = difference[difference.drop(columns=['gdot_names']).duplicated()==False]
print('Around',difference.shape[0],'GDOT traffic signals not in OSM')

1603 GDOT signals and 1549 OSM signals
Around 574 GDOT traffic signals not in OSM


## Add these new GDOT signals into the OSM network
Set a buffer for each signal and find all candidate nodes associated
- Filter OSM nodes to only consider links with a node with a degree higher than 2 and that's labelled as a road
- Buffer and intersect the GDOT signals with the filtered OSM nodes
- Check the road names for links attached to a candidate node


we set a buffer distance around each signal to find all candidate nodes associated with the traffic signal.
- Then, we remove links that are unlikely to be signalized intersections

In [10]:
from collections import Counter

#only consider road nodes (if there's a service road that connects to a signalized intersection this should still count them)
only_roads = links[links['link_type'].isin(['road'])].copy()
road_nodes = pd.Series(Counter(only_roads['osm_A'].tolist()+only_roads['osm_B'].tolist()))
road_nodes = set(road_nodes[road_nodes>2].index.tolist()) # and remove matches where degree is 2 or less
road_nodes = nodes[nodes['osm_N'].isin(road_nodes)]

buffer_ft = 100
gdot_buffered = buffer_signals(difference,buffer_ft)
candidate_signals = gpd.overlay(road_nodes,gdot_buffered,how="intersection")
# candidate_signals.explore()

# subset the links
candidate_signals0 = set(candidate_signals['osm_N'].tolist())
candidate_links = links[links['osm_A'].isin(candidate_signals0) | links['osm_B'].isin(candidate_signals0)].copy()
candidate_links = candidate_links[candidate_links['link_type']=='road']

In [11]:
# add the candidate nodes info to the candidate links so that we can check street name
from importlib import reload
reload(conflation_tools)
A = candidate_links.merge(candidate_signals,left_on='osm_A',right_on='osm_N')
name_check = A.apply(lambda row: any([conflation_tools.name_check(row['name0'],name,.6) for name in row['gdot_names']]), axis = 1)
# A[name_check==False].sample(30)
A = A[name_check]
A = set(A['osm_linkid'].tolist())

B = candidate_links.merge(candidate_signals,left_on='osm_B',right_on='osm_N')
name_check = B.apply(lambda row: any([conflation_tools.name_check(row['name0'],name,.6) for name in row['gdot_names']]), axis = 1)
# main_check = B.apply(lambda row: conflation_tools.name_check(row['name'],row['mainStreetName']),axis=1)
# side_check = B.apply(lambda row: conflation_tools.name_check(row['name'],row['sideStreetName']),axis=1)
B = B[name_check]
B = set(B['osm_linkid'].tolist())

candidate_links = links[links['osm_linkid'].isin(set.union(A,B))]
# candidate_links.explore()

In [12]:
_, turns_df = modeling_turns.create_pseudo_dual_graph(candidate_links,'osm_A','osm_B','osm_linkid','oneway')

In [13]:
#add signals ids back in
turns_df['signalized'] = turns_df['source_B'].isin(candidate_signals0)
turns_df = turns_df[turns_df['signalized'] == True]

# turns_df.drop(columns=['source','target'],inplace=True)

turns_df.to_pickle(config['network_fp']/'gdot_signals.pkl')